### Dataset: LastFM
### Algorithm: Time-Aware ItemKNN Incremental Ding 2005
### Padding unknown user/item: RecentPop
### Flow: Library Flow

In [1]:
k = 100
from streamsight.datasets.lastfm import LastFMDataset
from streamsight.settings import SlidingWindowSetting
dataset = LastFMDataset()

data = dataset.load()

setting_window = SlidingWindowSetting(
    background_t=1167550546,
    window_size=60 * 60 * 24 * 365, # 365 days
    top_K=k
)

setting_window.split(data)

INFO - streamsight package loaded.
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

DEBUG - LastFMDataset being initialized with 'data' as the base path.
DEBUG - LastFMDataset is initialized.
INFO - LastFMDataset is loading dataset...
DEBUG - Data zipfile is in memory and in dir specified.
DEBUG - LastFMDataset applying filters set.
DEBUG - 	interactions before preprocess: 186479
DEBUG - 	items before preprocess: 12523
DEBUG - 	users before preprocess: 1892
DEBUG - 	interactions after preprocess: 186479
DEBUG - 	items after preprocess: 12523
DEBUG - 	users after preprocess: 1892
INFO - LastFMDataset dataset loaded - Took 0.0908s
DEBUG - Splitting data...
DEBUG - Performing lt(t, 2147483647)
DEBUG - Performing lt(t, 1167550546)
DEBUG - Performing ge

  0%|          | 0/4 [00:00<?, ?it/s]

DEBUG - NPastInteractionTimestampSplitter(t=1167550546,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) - Updating split point to t=1167550546
DEBUG - Performing lt(t, 1199086546)
DEBUG - Performing ge(t, 1167550546)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=1167550546,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 1167550546 resulted in empty unlabelled testing samples.
DEBUG - NPastInteractionTimestampSplitter(t=1167550546,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) - Updating split point to t=1199086546
DEBUG - Performing lt(t, 1230622546)
DEBUG - Performing ge(t, 1199086546)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=1199086546,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 1199086546 re

5it [00:00, 61.69it/s]               

INFO - Finished split with window size 31536000 seconds. Number of splits: 5 in total.
INFO - SlidingWindowSetting data split - Took 0.104s
DEBUG - Checking split attribute and sizes.
DEBUG - Checking split attributes.
DEBUG - Split attributes are set.
DEBUG - Checking size of split sets.
DEBUG - Size of split sets are checked.
INFO - SlidingWindowSetting data split complete.


In [2]:
from streamsight.evaluators import EvaluatorStreamerBuilder

builder = EvaluatorStreamerBuilder()
builder.add_setting(setting_window)
builder.set_metric_K(k)
builder.add_metric("PrecisionK")
builder.add_metric("RecallK")
builder.add_metric("NDCGK")
evaluator = builder.build()

In [3]:
from streamsight.algorithms import TARSItemKNNDing
external_model = TARSItemKNNDing(K=k)
external_model_id = evaluator.register_algorithm(algorithm_name="LastFM_TA_ItemKNN_Inc_Ding")
print(external_model_id)

INFO - Registering algorithm name LastFM_TA_ItemKNN_Inc_Ding with ID: bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d registered
bdd640fb-0667-4ad1-9c80-317fa3b1799d


In [4]:
evaluator.start_stream()

DEBUG - Resetting data generators.
DEBUG - Data generators are reset.
DEBUG - Preparing evaluator for streaming
DEBUG - (user x item) shape defined is (187, 3034)
DEBUG - Shape of dataframe stored in matrix was (12302, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (12302, 5) after masking
DEBUG - Final (user x item) shape defined is (187, 3034)
DEBUG - Caching evaluation data for step 1
DEBUG - (user x item) shape defined is (187, 3034)
DEBUG - Shape of dataframe stored in matrix was (11440, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (11440, 5) after masking
DEBUG - Final (user x item) shape defined is (187, 3034)
DEBUG - (user x item) shape defined is (187, 3034)
DEBUG - Shape of dataframe stored in matrix was (11440, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (7817, 5) after masking
DEBUG - Final (user x item) shape defined is (457, 3034)
DEBUG - Data cached for step 1 complete


# Iteration 1

In [5]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
X after decay:  [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNDing missing similar items for 16 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Fitting TARSItemKNNDing complete - Took 0.219s


       interactionid  uid   iid          ts  tagID
0                  0    0     0  -428720400     39
1                  1    1     1  -420771600    103
2                  2    1     1  -420771600  10021
3                  3    2     2  -405133200   1474
4                  4    3     3   294357600    311
...              ...  ...   ...         ...    ...
12297          12297    3  1970  1164927600    102
12298          12298    3  1791  1164927600     81
12299          12299   99  3033  1164927600  11934
12300          12300    3  1791  1164927600     33
12301          12301    3   183  1164927600    387

[12302 rows x 5 columns]

In [6]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNDing missing similar items for 16 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Max user ID:  458
Max item ID:  0
X.shape:  (187, 3034)
Intended shape:  (458, 3034)
Predict frame:         interactionid  uid  iid          ts  tagID
12302          12302   24   -1  1167606000     76
12303          12303   60   -1  1167606000   1316
12304          12304   60   -1  1167606000   2664
12305          12305   76   -1  1167606000   1844
12306          12306    9   -1  1167606000   7263
...              ...  ...  ...      

       interactionid  uid  iid          ts  tagID
12302          12302   24   -1  1167606000     76
12303          12303   60   -1  1167606000   1316
12304          12304   60   -1  1167606000   2664
12305          12305   76   -1  1167606000   1844
12306          12306    9   -1  1167606000   7263
...              ...  ...  ...         ...    ...
32693          32693  457   -1  1196463600   4397
32694          32694  457   -1  1196463600   1911
32695          32695  200   -1  1196463600    855
32700          32700  315   -1  1196463600     81
32709          32709  457   -1  1196463600     79

[11440 rows x 5 columns]

In [7]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 934188 stored elements and shape (458, 3034)>
  Coords	Values
  (3, 3020)	0.7070985971276452
  (3, 2781)	0.5773435869335141
  (3, 2413)	0.7070985971276452
  (3, 1947)	0.7070985971276452
  (3, 2561)	0.7070985971276452
  (3, 623)	0.4082435653952528
  (3, 2734)	0.5773435869335141
  (3, 2093)	2.184846730402967e-27
  (3, 2193)	6.372643377110908e-94
  (3, 2018)	0.7070985971276452
  (3, 2184)	7.804862293324159e-94
  (3, 2944)	1.5609724586648317e-93
  (3, 826)	9.012278692077327e-94
  (3, 902)	0.999988425992905
  (3, 3024)	1.2844421840611593
  (3, 2983)	7.804862293324159e-94
  (3, 2975)	6.616762073716329e-14
  (3, 2297)	2.1212957913830017
  (3, 297)	5.518871053836701e-94
  (3, 2957)	0.7070985971276452
  (3, 2692)	1.2844421840611593
  (3, 2473)	7.804862293324159e-94
  (3, 1393)	7.804862293324159e-94
  (3, 2153)	6.372643377110908e-94
  (3, 2521)	0.8164871307905056
  :	:
  (457, 3009)	0.0
  (457, 3010)	0.0
  (457, 3011)	0.0
  (457, 3012

# Iteration 2

In [8]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (458, 5278)
DEBUG - Shape of dataframe stored in matrix was (20415, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (20415, 5) after masking
DEBUG - Final (user x item) shape defined is (458, 5278)
DEBUG - Caching evaluation data for step 2
DEBUG - (user x item) shape defined is (458, 5278)
DEBUG - Shape of dataframe stored in matrix was (21359, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (21359, 5) after masking
DEBUG - Final (user x item) shape defined is (458, 5278)
DEBUG - (user x item) shape defined is (458, 5278)
DEBUG - Shape of dataframe stored in matrix was (21359, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (15787, 5) after masking
DEBUG - Final (user x item) shape defined is (918, 5278)
DEBUG - Data cached for step 2 complete
X after decay:  [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 

       interactionid  uid   iid          ts  tagID
12302          12302   24  3034  1167606000     76
12303          12303   60  3035  1167606000   1316
12304          12304   60  3035  1167606000   2664
12305          12305   76  3036  1167606000   1844
12306          12306    9  1224  1167606000   7263
...              ...  ...   ...         ...    ...
32712          32712  340   512  1196463600    105
32713          32713  419  2309  1196463600     72
32714          32714  340   512  1196463600    570
32715          32715  131  5275  1196463600     47
32716          32716  131  5264  1196463600    187

[20415 rows x 5 columns]

In [9]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNDing missing similar items for 23 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Max user ID:  918
Max item ID:  0
X.shape:  (458, 5278)
Intended shape:  (918, 5278)
Predict frame:         interactionid  uid  iid          ts  tagID
32717          32717  181   -1  1199142000   3364
32718          32718  419   -1  1199142000     78
32719          32719  419   -1  1199142000    169
32720          32720  398   -1  1199142000     39
32721          32721  458   -1  1199142000     47
...              ...  ...  ...      

       interactionid  uid  iid          ts  tagID
32717          32717  181   -1  1199142000   3364
32718          32718  419   -1  1199142000     78
32719          32719  419   -1  1199142000    169
32720          32720  398   -1  1199142000     39
32721          32721  458   -1  1199142000     47
...              ...  ...  ...         ...    ...
72968          72968  780   -1  1228086000     25
72971          72971  862   -1  1228086000    515
72973          72973  823   -1  1228086000   7938
72974          72974   91   -1  1228086000   1835
72988          72988  460   -1  1228086000     83

[21359 rows x 5 columns]

In [10]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2800379 stored elements and shape (918, 5278)>
  Coords	Values
  (3, 3088)	5.402563610001634e-14
  (3, 3727)	6.616762073716329e-14
  (3, 4469)	0.7070985971276452
  (3, 4968)	5.746612993258902e-54
  (3, 3807)	6.616762073716329e-14
  (3, 4769)	6.616762073716329e-14
  (3, 2454)	1.8511795943192944e-80
  (3, 3535)	1.8511795943192944e-80
  (3, 3921)	1.8511795943192944e-80
  (3, 4360)	1.8511795943192944e-80
  (3, 5020)	7.804862293324159e-94
  (3, 4882)	1.60316855564788e-80
  (3, 4466)	3.0266821850688794e-120
  (3, 4111)	2.621183661451443e-120
  (3, 1706)	9.926875755274419e-133
  (3, 2594)	8.543694328899557e-133
  (3, 2147)	1.479811266199211e-132
  (3, 2172)	1.479811266199211e-132
  (3, 2175)	1.479811266199211e-132
  (3, 2176)	1.479811266199211e-132
  (3, 2190)	1.479811266199211e-132
  (3, 2194)	1.479811266199211e-132
  (3, 2390)	1.479811266199211e-132
  (3, 2397)	1.479811266199211e-132
  (3, 3070)	1.479811266199211e-132
  :	:
  (91

# Iteration 3

In [11]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (918, 8205)
DEBUG - Shape of dataframe stored in matrix was (40273, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (40273, 5) after masking
DEBUG - Final (user x item) shape defined is (918, 8205)
DEBUG - Caching evaluation data for step 3
DEBUG - (user x item) shape defined is (918, 8205)
DEBUG - Shape of dataframe stored in matrix was (23605, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (23605, 5) after masking
DEBUG - Final (user x item) shape defined is (918, 8205)
DEBUG - (user x item) shape defined is (918, 8205)
DEBUG - Shape of dataframe stored in matrix was (23605, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (19148, 5) after masking
DEBUG - Final (user x item) shape defined is (1369, 8205)
DEBUG - Data cached for step 3 complete
X after decay:  [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0.

       interactionid  uid   iid          ts  tagID
32717          32717  181  1332  1199142000   3364
32718          32718  419  5278  1199142000     78
32719          32719  419  5278  1199142000    169
32720          32720  398  1080  1199142000     39
32721          32721  458  5279  1199142000     47
...              ...  ...   ...         ...    ...
72985          72985  639   744  1228086000   8504
72986          72986  502  8204  1228086000    109
72987          72987  502  8200  1228086000    724
72988          72988  460  2139  1228086000     83
72989          72989  502  8200  1228086000    109

[40273 rows x 5 columns]

In [12]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNDing missing similar items for 47 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 1.02646356e-146 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 ...
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.0000000

        interactionid   uid  iid          ts  tagID
72990           72990   918   -1  1230764400    176
72991           72991   412   -1  1230764400    275
72992           72992   918   -1  1230764400    227
72993           72993   460   -1  1230764400     15
72994           72994   196   -1  1230764400    187
...               ...   ...  ...         ...    ...
116338         116338   456   -1  1259622000    693
116341         116341   207   -1  1259622000    387
116347         116347   207   -1  1259622000    191
116348         116348  1339   -1  1259622000   4228
116351         116351   456   -1  1259622000    432

[23605 rows x 5 columns]

In [13]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4521420 stored elements and shape (1369, 8205)>
  Coords	Values
  (1, 177)	1.3251587564408734e-146
  (1, 862)	1.3012800662622535e-146
  (1, 1122)	1.5396953384329293e-146
  (1, 1272)	1.2731713510051326e-146
  (1, 1526)	1.405543114242111e-146
  (1, 1698)	1.5301615295002554e-146
  (1, 1861)	1.405543114242111e-146
  (1, 2203)	1.3251587564408731e-146
  (1, 2527)	1.3251587564408731e-146
  (1, 2889)	1.3251587564408731e-146
  (1, 2904)	1.3251587564408731e-146
  (1, 3066)	1.6229813907306147e-146
  (1, 3322)	1.6229813907306147e-146
  (1, 3376)	1.3251587564408731e-146
  (1, 3495)	1.6229813907306147e-146
  (1, 3676)	1.3251587564408731e-146
  (1, 3771)	1.3251587564408731e-146
  (1, 4121)	1.3251587564408731e-146
  (1, 4295)	1.6229813907306147e-146
  (1, 4957)	1.6229813907306147e-146
  (1, 5353)	1.6229813907306147e-146
  (1, 5990)	1.6229813907306147e-146
  (1, 6209)	1.6229813907306147e-146
  (1, 6624)	1.6229813907306147e-146
  (1, 6733)	1.

# Iteration 4

In [14]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (1369, 10380)
DEBUG - Shape of dataframe stored in matrix was (43366, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (43366, 5) after masking
DEBUG - Final (user x item) shape defined is (1369, 10380)
DEBUG - Caching evaluation data for step 4
DEBUG - (user x item) shape defined is (1369, 10380)
DEBUG - Shape of dataframe stored in matrix was (27243, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (27243, 5) after masking
DEBUG - Final (user x item) shape defined is (1369, 10380)
DEBUG - (user x item) shape defined is (1369, 10380)
DEBUG - Shape of dataframe stored in matrix was (27243, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (23382, 5) after masking
DEBUG - Final (user x item) shape defined is (1821, 10380)
DEBUG - Data cached for step 4 complete
X after decay:  [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 

        interactionid   uid   iid          ts  tagID
72990           72990   918  2027  1230764400    176
72991           72991   412  1223  1230764400    275
72992           72992   918  5332  1230764400    227
72993           72993   460  2144  1230764400     15
72994           72994   196  1446  1230764400    187
...               ...   ...   ...         ...    ...
116351         116351   456  1306  1259622000    432
116352         116352   739   376  1259622000     16
116353         116353  1301  1095  1259622000    847
116354         116354  1115   736  1259622000    134
116355         116355   428  3980  1259622000   8304

[43366 rows x 5 columns]

In [15]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNDing missing similar items for 74 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [0.000000e+00 3.426444e-67 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 ...
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]]
Max user ID:  1821
Max item ID:  0
X.shape:  (1369, 10380)
Intend

        interactionid   uid  iid          ts  tagID
116356         116356   220   -1  1262300400    386
116357         116357   220   -1  1262300400    387
116358         116358   220   -1  1262300400    147
116359         116359   220   -1  1262300400    386
116360         116360   220   -1  1262300400     72
...               ...   ...  ...         ...    ...
171349         171349  1815   -1  1291158000     73
171350         171350  1815   -1  1291158000     81
171351         171351  1801   -1  1291158000    275
171352         171352  1815   -1  1291158000    134
171353         171353  1698   -1  1291158000     16

[27243 rows x 5 columns]

In [16]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5900750 stored elements and shape (1821, 10380)>
  Coords	Values
  (1, 1)	3.4264440039660706e-67
  (1, 2527)	3.9883893796597683e-305
  (1, 3554)	4.2041315451169684e-305
  (1, 4295)	4.2041315451169684e-305
  (1, 4957)	4.2041315451169684e-305
  (1, 5121)	4.2041315451169684e-305
  (1, 5990)	4.2041315451169684e-305
  (1, 6209)	4.2041315451169684e-305
  (1, 6380)	4.2041315451169684e-305
  (1, 6510)	4.2041315451169684e-305
  (1, 6624)	4.2041315451169684e-305
  (1, 7297)	4.2041315451169684e-305
  (1, 7680)	4.2041315451169684e-305
  (1, 8439)	4.2041315451169684e-305
  (1, 1070)	3.726564317122633e-305
  (1, 1272)	3.586942704014545e-305
  (1, 1597)	3.485881726112522e-305
  (1, 2583)	4.269315766333973e-305
  (1, 2585)	8.156490797251378e-305
  (1, 2629)	4.269315766333973e-305
  (1, 3079)	3.485881726112522e-305
  (1, 3192)	4.4093303633899276e-305
  (1, 3377)	3.485881726112522e-305
  (1, 3492)	5.456267950778332e-305
  (1, 3837)	5.03143688

# Iteration 5

In [17]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (1821, 12146)
DEBUG - Shape of dataframe stored in matrix was (54998, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (54998, 5) after masking
DEBUG - Final (user x item) shape defined is (1821, 12146)
DEBUG - Caching evaluation data for step 5
DEBUG - (user x item) shape defined is (1821, 12146)
DEBUG - Shape of dataframe stored in matrix was (10224, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (10224, 5) after masking
DEBUG - Final (user x item) shape defined is (1821, 12146)
DEBUG - (user x item) shape defined is (1821, 12146)
DEBUG - Shape of dataframe stored in matrix was (10224, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (9342, 5) after masking
DEBUG - Final (user x item) shape defined is (1890, 12146)
DEBUG - Data cached for step 5 complete
X after decay:  [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0

        interactionid   uid    iid          ts  tagID
116356         116356   220   1473  1262300400    386
116357         116357   220    528  1262300400    387
116358         116358   220   1554  1262300400    147
116359         116359   220   2352  1262300400    386
116360         116360   220  10380  1262300400     72
...               ...   ...    ...         ...    ...
171349         171349  1815  10021  1291158000     73
171350         171350  1815  10021  1291158000     81
171351         171351  1801     30  1291158000    275
171352         171352  1815  10021  1291158000    134
171353         171353  1698    202  1291158000     16

[54998 rows x 5 columns]

In [18]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNDing missing similar items for 85 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[1.72031189e+00 0.00000000e+00 0.00000000e+00 ... 4.47208419e-01
  0.00000000e+00 0.00000000e+00]
 [2.62923464e-14 6.96915359e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.82139683e-14 ... 0.00000000e+00
  1.03311174e-67 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000

        interactionid   uid  iid          ts  tagID
171354         171354   853   -1  1293836400     24
171355         171355   853   -1  1293836400    130
171356         171356  1801   -1  1293836400    275
171357         171357  1186   -1  1293836400    424
171358         171358  1404   -1  1293836400     39
...               ...   ...  ...         ...    ...
186445         186445  1237   -1  1304940155    389
186446         186446  1237   -1  1304940165     80
186447         186447  1237   -1  1304940174    481
186448         186448  1237   -1  1304940176    121
186449         186449  1237   -1  1304940176    121

[10224 rows x 5 columns]

In [19]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2212176 stored elements and shape (1892, 12146)>
  Coords	Values
  (0, 9973)	4.678757331822778e-14
  (0, 11110)	4.678757331822778e-14
  (0, 11638)	4.678757331822778e-14
  (0, 11978)	6.616762073716329e-14
  (0, 12139)	6.616762073716329e-14
  (0, 1754)	9.357514663645556e-14
  (0, 4041)	9.357514663645556e-14
  (0, 10864)	4.678757331822778e-14
  (0, 11124)	4.678757331822778e-14
  (0, 11373)	4.678757331822778e-14
  (0, 11526)	4.678757331822778e-14
  (0, 11554)	4.678757331822778e-14
  (0, 11788)	9.357514663645556e-14
  (0, 6859)	4.678757331822778e-14
  (0, 9471)	4.678757331822778e-14
  (0, 9472)	4.678757331822778e-14
  (0, 10192)	4.678757331822778e-14
  (0, 11065)	4.678757331822778e-14
  (0, 10127)	9.77093161919838e-28
  (0, 10702)	9.77093161919838e-28
  (0, 10850)	9.77093161919838e-28
  (0, 11000)	9.77093161919838e-28
  (0, 11890)	1.3818184012890457e-27
  (0, 7340)	1.5449199389211943e-27
  (0, 7342)	1.5449199389211943e-27
  :	:
 

# evaluate metrics

In [20]:
evaluator.get_all_algorithm_status()

{'LastFM_TA_ItemKNN_Inc_Ding_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.COMPLETED: 'COMPLETED'>}

In [21]:
evaluator.metric_results("macro")

macro_score  \
Algorithm                                          Metric                        
LastFM_TA_ItemKNN_Inc_Ding_bdd640fb-0667-4ad1-9... NDCGK_100          0.113237   
                                                   PrecisionK_100     0.019715   
                                                   RecallK_100        0.232261   

                                                                   num_window  
Algorithm                                          Metric                      
LastFM_TA_ItemKNN_Inc_Ding_bdd640fb-0667-4ad1-9... NDCGK_100                5  
                                                   PrecisionK_100           5  
                                                   RecallK_100              5

In [22]:
evaluator.metric_results("micro")


micro_score  \
Algorithm                                          Metric                       
LastFM_TA_ItemKNN_Inc_Ding_bdd640fb-0667-4ad1-9... NDCGK_100          0.11362   
                                                   PrecisionK_100    0.019622   
                                                   RecallK_100       0.230125   

                                                                   num_user  
Algorithm                                          Metric                    
LastFM_TA_ItemKNN_Inc_Ding_bdd640fb-0667-4ad1-9... NDCGK_100           3282  
                                                   PrecisionK_100      3282  
                                                   RecallK_100         3282

In [23]:
evaluator.metric_results("user")

user_id  \
Algorithm                                          Timestamp    Metric                   
LastFM_TA_ItemKNN_Inc_Ding_bdd640fb-0667-4ad1-9... t=1167550546 PrecisionK_100       3   
                                                                PrecisionK_100       4   
                                                                PrecisionK_100       8   
                                                                PrecisionK_100       9   
                                                                PrecisionK_100      12   
...                                                                                ...   
                                                   t=1293694546 NDCGK_100         1885   
                                                                NDCGK_100         1886   
                                                                NDCGK_100         1887   
                                                                NDCGK_100         1888   
                                                                NDCGK_100         1889   

                                                                                   score  
Algorithm                                          Timestamp    Metric                    
LastFM_TA_ItemKNN_Inc_Ding_bdd640fb-0667-4ad1-9... t=1167550546 PrecisionK_100       0.0  
                                                                PrecisionK_100      0.01  
                                                                PrecisionK_100       0.0  
                                                                PrecisionK_100      0.13  
                                                                PrecisionK_100       0.0  
...                                                                                  ...  
                                                   t=1293694546 NDCGK_100       0.176507  
                                                                NDCGK_100            0.0  
                                                                NDCGK_100       0.348502  
                                                                NDCGK_100       0.144808  
                                                                NDCGK_100            0.0  

[9846 rows x 2 columns]

In [24]:
evaluator.metric_results("window")

window_score  \
Algorithm                                          Timestamp    Metric                         
LastFM_TA_ItemKNN_Inc_Ding_bdd640fb-0667-4ad1-9... t=1167550546 PrecisionK_100      0.024382   
                                                                RecallK_100         0.258612   
                                                                NDCGK_100           0.120425   
                                                   t=1199086546 PrecisionK_100      0.018799   
                                                                RecallK_100         0.199030   
                                                                NDCGK_100           0.099539   
                                                   t=1230622546 PrecisionK_100      0.020584   
                                                                RecallK_100         0.215815   
                                                                NDCGK_100           0.109082   
                                                   t=1262158546 PrecisionK_100      0.020608   
                                                                RecallK_100         0.247939   
                                                                NDCGK_100           0.126494   
                                                   t=1293694546 PrecisionK_100      0.014202   
                                                                RecallK_100         0.239911   
                                                                NDCGK_100           0.110643   

                                                                                num_user  
Algorithm                                          Timestamp    Metric                    
LastFM_TA_ItemKNN_Inc_Ding_bdd640fb-0667-4ad1-9... t=1167550546 PrecisionK_100       356  
                                                                RecallK_100          356  
                                                                NDCGK_100            356  
                                                   t=1199086546 PrecisionK_100       658  
                                                                RecallK_100          658  
                                                                NDCGK_100            658  
                                                   t=1230622546 PrecisionK_100       805  
                                                                RecallK_100          805  
                                                                NDCGK_100            805  
                                                   t=1262158546 PrecisionK_100       937  
                                                                RecallK_100          937  
                                                                NDCGK_100            937  
                                                   t=1293694546 PrecisionK_100       526  
                                                                RecallK_100          526  
                                                                NDCGK_100            526